In [32]:
import time
import random
import math
people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]

# Место назначения – аэропорт Ла Гардиа в Нью-Йорке
destination='LGA'

In [33]:
flights={}
"""
Для каждого рейса в файле через запятую указаны аэропорт отправления,
аэропорт назначения, время отправления, время прибытия и цена билета:
"""
for line in open('data/schedule.txt'):
    origin,dest,depart,arrive,price=line.strip().split(',')
    flights.setdefault((origin,dest),[])
    # Добавить информацию о возможном рейсе в список
    flights[(origin,dest)].append((depart,arrive,int(price)))


In [34]:
def getminutes(t):
    x=time.strptime(t,'%H:%M')
    return x[3]*60+x[4]

In [35]:
def printschedule(r):
    for d in range(int(len(r)/2)):
        name=people[d][0]
        origin=people[d][1]
        out=flights[(origin,destination)][r[d]]
        ret=flights[(destination,origin)][r[d+1]]
        print ('{0} {1} {2}-{3} ${4} | {5}-{6} ${7}'.format(name,origin,
                                                     out[0],out[1],out[2],
                                                     ret[0],ret[1],ret[2]))

In [36]:
s=[1,4,3,2,7,3,6,3,2,4,5,3]
printschedule(s)

Seymour BOS 8:04-10:11 $95 | 12:08-14:05 $142
Franny DAL 12:19-15:25 $342 | 10:51-14:16 $256
Zooey CAK 10:53-13:36 $189 | 9:58-12:56 $249
Walt MIA 9:15-12:29 $225 | 16:50-19:26 $304
Buddy ORD 16:43-19:00 $246 | 10:33-13:11 $132
Les OMA 11:08-13:07 $175 | 15:07-17:21 $129


In [37]:
def schedulecost(sol):
    totalprice=0
    latestarrival=0
    earliestdep=24*60

    for d in range(int(len(sol)/2)):
        # Get the inbound and outbound flights
        origin=people[d][1]
        outbound=flights[(origin,destination)][int(sol[d])]
        returnf=flights[(destination,origin)][int(sol[d+1])]
    
        # Total price is the price of all outbound and return flights
        totalprice+=outbound[2]
        totalprice+=returnf[2]
    
        # Track the latest arrival and earliest departure
        if latestarrival<getminutes(outbound[1]): latestarrival=getminutes(outbound[1])
        if earliestdep>getminutes(returnf[0]): earliestdep=getminutes(returnf[0])
      
      # Every person must wait at the airport until the latest person arrives.
      # They also must arrive at the same time and wait for their flights.
        totalwait=0  
    for d in range(int(len(sol)/2)):
        origin=people[d][1]
        outbound=flights[(origin,destination)][int(sol[d])]
        returnf=flights[(destination,origin)][int(sol[d+1])]
        totalwait+=latestarrival-getminutes(outbound[1])
        totalwait+=getminutes(returnf[0])-earliestdep  

        # Для этого решения требуется оплачивать дополнительный день аренды?
        # Если да, это обойдется в лишние $50!
        if latestarrival>earliestdep: totalprice+=50
        return int(totalprice+totalwait)

In [38]:
schedulecost(s)

3193

In [39]:
def randomoptimize(domain,costf):
    best=999999999
    bestr=None
    for i in range(0,1000):
        # Create a random solution
        r=[int(random.randint(domain[i][0],domain[i][1]))
        for i in range(len(domain))]
        # Get the cost
        cost=costf(r)
        # Compare it to the best one so far
        if cost<best:
            best=cost
        bestr=r
    return r

In [40]:
domain=[(0,8)]*(len(people)*2)

In [41]:
 s=randomoptimize(domain,schedulecost)

In [42]:
schedulecost(s)

3702

In [43]:
printschedule(s)

Seymour BOS 8:04-10:11 $95 | 8:23-10:28 $149
Franny DAL 7:53-11:37 $433 | 14:20-17:32 $332
Zooey CAK 13:40-15:38 $137 | 18:17-21:04 $259
Walt MIA 18:23-21:35 $134 | 18:07-21:30 $355
Buddy ORD 18:48-21:45 $246 | 6:03-8:43 $219
Les OMA 6:11-8:31 $249 | 9:31-11:43 $210


In [48]:
def hillclimb(domain,costf):
     # Выбрать случайное решение
     sol=[random.randint(domain[i][0],domain[i][1])
       for i in range(len(domain))]:
         # Главный цикл
         while 1:
           # Создать список соседних решений
           neighbors=[]
           for j in range(len(domain)):
             # Отходим на один шаг в каждом направлении
             if sol[j]>domain[j][0]:
               neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
             if sol[j]<domain[j][1]:
               neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])
       # Ищем наилучшее из соседних решений
        current=costf(sol)
           best=current
       for j in range(len(neighbors)):
         cost=costf(neighbors[j])
         if cost<best:
           best=cost
           sol=neighbors[j]
       # Если улучшения нет, мы достигли дна
       if best==current:
         break
     return sol

In [49]:
schedulecost(s)
domain=[(0,8)]*(len(people)*2)
s=[1,4,3,2,7,3,6,3,2,4,5,3]
s=hillclimb(domain,schedulecost)

IndexError: list index out of range

In [17]:
schedulecost(s)

2811

In [18]:
printschedule(s)

Seymour BOS 17:11-18:30 $108 | 10:33-12:03 $74
Franny DAL 10:30-14:57 $290 | 6:09-9:49 $414
Zooey CAK 6:08-8:06 $224 | 13:37-15:33 $142
Walt MIA 14:01-17:24 $338 | 12:37-15:05 $170
Buddy ORD 12:44-14:17 $134 | 6:03-8:43 $219
Les OMA 6:11-8:31 $249 | 15:07-17:21 $129


In [19]:
def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
  # Инициализировать переменные случайным образом
  vec=[int(random.randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
  
  while T>0.1:
    # Выбрать один из индексов
    i=random.randint(0,len(domain)-1)

    # Выбрать направление изменения
    dir=random.randint(-step,step)

    # Создать новый список, в котором одно значение изменено
    vecb=vec[:]
    vecb[i]+=dir
    if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
    elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

    # Вычислить текущую и новую стоимость
    ea=costf(vec)
    eb=costf(vecb)
    p=pow(math.e,(-eb-ea)/T)

    print (vec,ea)


    # Новое решение лучше? Если нет, метнем кости
    if (eb<ea or random.random()<p):
      vec=vecb      

    # Уменьшить температуру
    T=T*cool
  return vec


In [20]:
annealingoptimize(domain,schedulecost)

[1, 7, 3, 6, 8, 5, 1, 4, 7, 5, 4, 0] 4120
[1, 7, 3, 6, 8, 5, 1, 4, 7, 5, 4, 0] 4120
[1, 7, 3, 6, 8, 5, 1, 4, 7, 5, 4, 0] 4120
[1, 7, 3, 6, 8, 5, 2, 4, 7, 5, 4, 0] 4107
[1, 7, 3, 6, 8, 5, 2, 4, 7, 5, 4, 0] 4107
[1, 7, 3, 6, 8, 5, 2, 4, 6, 5, 4, 0] 4107
[1, 7, 3, 6, 8, 5, 2, 4, 6, 5, 4, 0] 4107
[1, 7, 3, 6, 8, 5, 3, 4, 6, 5, 4, 0] 3988
[0, 7, 3, 6, 8, 5, 3, 4, 6, 5, 4, 0] 4087
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 6, 4, 4, 6, 5, 4, 0] 4034
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 8, 5, 4, 4, 6, 5, 4, 0] 4150
[0, 7, 3, 6, 7, 5, 4, 4, 6, 5, 4, 0] 4042
[0, 7, 3, 6, 7, 5, 4, 4, 6, 5, 4, 

[3, 5, 3, 5, 6, 7, 3, 4, 6, 5, 4, 0]

In [21]:
schedulecost(s)

2811

In [22]:
printschedule(s)

Seymour BOS 17:11-18:30 $108 | 10:33-12:03 $74
Franny DAL 10:30-14:57 $290 | 6:09-9:49 $414
Zooey CAK 6:08-8:06 $224 | 13:37-15:33 $142
Walt MIA 14:01-17:24 $338 | 12:37-15:05 $170
Buddy ORD 12:44-14:17 $134 | 6:03-8:43 $219
Les OMA 6:11-8:31 $249 | 15:07-17:21 $129


In [23]:
def geneticoptimize(domain,costf,popsize=50,step=1,mutprod=0.2,elite=0.2,maxiter=100):    
    # Операция мутации
    def mutate(vec):
        i=random.randint(0,len(domain)-1)
        if random.random()<0.5 and vec[i]>domain[i][0]:
            return (vec[0:i]+[vec[i]-step]+vec[i+1:])
        elif vec[i]<domain[i][1]:
            return (vec[0:i]+[vec[i]+step]+vec[i+1:])
        
    # Операция скрещивания
    def crossover(r1,r2):
        i=random.randint(1,len(domain)-2)
        return (r1[0:i]+r2[i:])
    
    #Строим начальную популяцию
    pop=[]
    for i in range(popsize):
        vec=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
        pop.append(vec)
                            
    # Сколько брать победителей из каждой популяции?
    topelite=int(elite*popsize)
    
    
    # Главный цикл
    for i in range(maxiter):
        scores=[(costf(v),v) for v in pop]
        scores.sort()
        ranked=[v for (s,v) in scores]
        # Сначала включаем только победителей
        pop=ranked[0:topelite]
        # Добавляем особей, полученных мутацией и скрещиванием победивших
        # родителей
        while len(pop)<popsize:
            if random.random()<mutprod:
                # Мутация
                c=random.randint(0,topelite)
                pop.append(mutate(ranked[c]))
            else:
                # Скрещивание
                c1=random.randint(0,topelite)
                c2=random.randint(0,topelite)
                pop.append(crossover(ranked[c1],ranked[c2]))
        # Печатаем полученные к текущему моменту наилучшие результаты
        print (scores[0][0])
    return scores[0][1]

In [24]:
geneticoptimize(domain,schedulecost)

2470
2320
2320
2278


TypeError: object of type 'NoneType' has no len()

In [25]:
printschedule(s)

Seymour BOS 17:11-18:30 $108 | 10:33-12:03 $74
Franny DAL 10:30-14:57 $290 | 6:09-9:49 $414
Zooey CAK 6:08-8:06 $224 | 13:37-15:33 $142
Walt MIA 14:01-17:24 $338 | 12:37-15:05 $170
Buddy ORD 12:44-14:17 $134 | 6:03-8:43 $219
Les OMA 6:11-8:31 $249 | 15:07-17:21 $129


In [26]:
GENTETIC = geneticoptimize(domain,schedulecost)

2528
2528
2306
2305
2214
2162
2126
2126
2126
2126
2126
2126
2126
2126
2126
2126
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120
2120


In [27]:
TEMP =  annealingoptimize(domain,schedulecost);

[8, 6, 1, 6, 3, 8, 5, 7, 3, 5, 8, 0] 3307
[8, 6, 1, 6, 3, 8, 5, 7, 3, 5, 8, 0] 3307
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 3, 8, 6, 7, 3, 5, 8, 0] 3210
[8, 6, 1, 6, 4, 8, 6, 7, 3, 5, 8, 

In [29]:
HILL = hillclimb(domain,schedulecost)

IndexError: list index out of range

In [30]:
 RAND =randomoptimize(domain,schedulecost)

In [31]:
print("Цена от Генетического Алгоритма \t {0}".format(schedulecost(GENTETIC)))
print("Цена от Алгоритма Отжига \t {0}".format(schedulecost(TEMP)))
print("Цена от Алгоритма Спуска с Горы \t {0}".format(schedulecost(HILL)))
print("Цена от Случайного поиска \t {0}".format(schedulecost(RAND)))

Цена от Генетического Алгоритма 	 2120
Цена от Алгоритма Отжига 	 2322


NameError: name 'HILL' is not defined